# SQL Databases

In [1]:
import sqlite3
import os

os.makedirs('data/databases', exist_ok=True)

In [2]:
## create a sample SQLite database
conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()

In [3]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS employees 
                   (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects 
                   (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [5]:
# Insert sample data 
employees_data = [
    (1, 'Alice Johnson', 'Software Engineer', 'Development', 90000),
    (2, 'Bob Smith', 'Data Scientist', 'Analytics', 95000),
    (3, 'Charlie Brown', 'Product Manager', 'Product', 105000),
    (4, 'Diana Prince', 'UX Designer', 'Design', 85000)
]

projects_data = [
    (1, 'Project Alpha', 'In Progress', 150000, 3),
    (2, 'Project Beta', 'Completed', 200000, 1),
    (3, 'Project Gamma', 'Planned', 300000, 4),
    (4, 'Project Delta', 'In Progress', 250000, 2)
]

In [6]:
cursor.executemany('INSERT INTO employees VALUES (?,?,?,?,?)', employees_data)
cursor.executemany('INSERT INTO projects VALUES (?,?,?,?,?)', projects_data)

In [7]:
cursor.execute('SELECT * FROM employees')

In [8]:
conn.commit()
conn.close()

# Database Content Extraction

In [9]:
from langchain_community.document_loaders import SQLDatabaseLoader
from langchain_community.utilities import SQLDatabase

d:\RAG\RAG-Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Method 1 SQLDatabase Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get database info
print(f"Tables: {db.get_usable_table_names()}")
print("\nDDL")
print(f"Table Info: {db.get_table_info()}")

Tables: ['employees', 'projects']

DDL
Table Info: 
CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	Alice Johnson	Software Engineer	Development	90000.0
2	Bob Smith	Data Scientist	Analytics	95000.0
3	Charlie Brown	Product Manager	Product	105000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	Project Alpha	In Progress	150000.0	3
2	Project Beta	Completed	200000.0	1
3	Project Gamma	Planned	300000.0	4
*/


In [13]:
from typing import List
from langchain_core.documents import Document

print("\nCustom SQL Processing:\n")

def sql_to_docs(db_path: str) -> List[Document]:
    """Convert SQL DB to a list of Documents with context"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    documents = []
    # Strategy 1 " Create documents for each table"
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    for table in tables:
        table_name = table[0]

        # Get Table Schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        # print(columns)
        column_names = [col[1] for col in columns]

        # Get Table Data
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()

        # Create Table Overview Document
        table_content = f"Table: {table_name}\nColumns: {', '.join(column_names)}\nNumber of rows: {len(rows)}\n\nSample Data:\n"

        for row in rows:
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
            
        doc = Document(page_content=table_content, metadata={"source": db_path, "table": table_name, 'num_records': len(rows), 'data_type': 'sql_table'})

        documents.append(doc)
    
    # Strategy 2: Create relationship documents
    cursor.execute("""SELECT e.name, e.role, p.name as project_name, p.status
                      FROM employees e
                      JOIN projects p ON e.id = p.lead_id;""")
    relations = cursor.fetchall()
    relation_content = "Employee-Project Relationships:\n"
    for relation in relations:
        relation_content += f"Employee: {relation[0]}, Role: {relation[1]}, Project: {relation[2]}, Status: {relation[3]}\n"

    rel_doc = Document(page_content=relation_content, metadata={"source": db_path, "data_type": "sql_relationships", 'query': 'employee_project_join'})
    documents.append(rel_doc)
    
    conn.close()
    return documents


Custom SQL Processing:



In [14]:
sql_to_docs('data/databases/company.db')

[Document(metadata={'source': 'data/databases/company.db', 'table': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nNumber of rows: 4\n\nSample Data:\n{'id': 1, 'name': 'Alice Johnson', 'role': 'Software Engineer', 'department': 'Development', 'salary': 90000.0}\n{'id': 2, 'name': 'Bob Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 95000.0}\n{'id': 3, 'name': 'Charlie Brown', 'role': 'Product Manager', 'department': 'Product', 'salary': 105000.0}\n{'id': 4, 'name': 'Diana Prince', 'role': 'UX Designer', 'department': 'Design', 'salary': 85000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nNumber of rows: 4\n\nSample Data:\n{'id': 1, 'name': 'Project Alpha', 'status': 'In Progress', 'budget': 150000.0, 'lead_id': 3}\n{'